In [1]:
import json
import pandas as pd
import numpy as np
import scipy.sparse.linalg

In [16]:
playlist = pd.read_json('res/val.json',encoding='utf-8')

In [21]:
playlist

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [3]:
song_df = pd.read_json('res/song_meta.json',encoding='utf-8')
song_df.rename(columns = {'id':'songs'},inplace = True)

In [4]:
answers = []

In [5]:
for i in range(0,len(playlist),100):
    playlist_test = playlist.loc[i:i+99]
    playlist_exp = playlist_test.explode('songs')
    playlist_exp['contain'] = 1
    plylst_df=playlist_exp.pivot_table('contain',index='id',columns='songs').fillna(0)
    matrix = plylst_df.as_matrix()
    user_contain_mean = np.mean(matrix, axis=1)
    matrix_user_mean = matrix - user_contain_mean.reshape(-1,1)
    U, sigma, Vt = scipy.sparse.linalg.svds(matrix_user_mean, k=12)
    sigma = np.diag(sigma)
    svd_user_predicted_contain = np.dot(np.dot(U, sigma),Vt) + user_contain_mean.reshape(-1,1)
    df_svd_preds = pd.DataFrame(svd_user_predicted_contain, index = plylst_df.index, columns = plylst_df.columns)
    for j in list(playlist_test['id']):
        songs_list = []
        if j not in df_svd_preds.index:
            songs_list = list(song_df.loc[:99,'songs'])
        else:
            songs_list = playlist_test[playlist_test['id'] == j]['songs'].iloc[0]
            songs_list_sub = df_svd_preds.loc[j].sort_values(ascending=False)
            for t in songs_list_sub.index[:][:100]:
                if t in songs_list or len(songs_list) == 100:
                    pass
                else:
                    songs_list.append(t)
        answers.append({
            "id": int(j),
            "songs": songs_list,
            "tags": ['기분전환','감성','드라이브','휴식','발라드','잔잔한','힐링','새벽','카페','이별']
        })

C:\Users\analysis\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [6]:
from arena_util import write_json
write_json(answers, 'cos/results.json')

In [7]:
from arena_util import load_json

def _idcg(l):
    return sum((1.0 / np.log(i + 2) for i in range(l)))

_idcgs = [_idcg(i) for i in range(101)]

def _ndcg(gt, rec):
    dcg = 0.0
    for i, r in enumerate(rec):
        if r in gt:
            dcg += 1.0 / np.log(i + 2)

    return dcg / _idcgs[len(gt)]


def _eval(gt_fname, rec_fname):
    gt_playlists = load_json(gt_fname)
    gt_dict = {g["id"]: g for g in gt_playlists}
    rec_playlists = load_json(rec_fname)

    gt_ids = set([g["id"] for g in gt_playlists])
    rec_ids = set([r["id"] for r in rec_playlists])

    if gt_ids != rec_ids:
        raise Exception("결과의 플레이리스트 수가 올바르지 않습니다.")

    rec_song_counts = [len(p["songs"]) for p in rec_playlists]
    rec_tag_counts = [len(p["tags"]) for p in rec_playlists]

    if set(rec_song_counts) != set([100]):
        raise Exception("추천 곡 결과의 개수가 맞지 않습니다.")

    if set(rec_tag_counts) != set([10]):
        raise Exception("추천 태그 결과의 개수가 맞지 않습니다.")

    rec_unique_song_counts = [len(set(p["songs"])) for p in rec_playlists]
    rec_unique_tag_counts = [len(set(p["tags"])) for p in rec_playlists]

    if set(rec_unique_song_counts) != set([100]):
        raise Exception("한 플레이리스트에 중복된 곡 추천은 허용되지 않습니다.")

    if set(rec_unique_tag_counts) != set([10]):
        raise Exception("한 플레이리스트에 중복된 태그 추천은 허용되지 않습니다.")

    music_ndcg = 0.0
    tag_ndcg = 0.0

    for rec in rec_playlists:
        gt = gt_dict[rec["id"]]
        music_ndcg += _ndcg(gt["songs"], rec["songs"][:100])
        tag_ndcg += _ndcg(gt["tags"], rec["tags"][:10])

    music_ndcg = music_ndcg / len(rec_playlists)
    tag_ndcg = tag_ndcg / len(rec_playlists)
    score = music_ndcg * 0.85 + tag_ndcg * 0.15

    return music_ndcg, tag_ndcg, score

In [8]:
gt_fname="res/val.json"
rec_fname="arena_data/cos/results.json"

try:
    music_ndcg, tag_ndcg, score = _eval(gt_fname, rec_fname)
    print(f"Music nDCG: {music_ndcg:.6}")
    print(f"Tag nDCG: {tag_ndcg:.6}")
    print(f"Score: {score:.6}")
except Exception as e:
    print(e)

float division by zero


In [9]:
playlist

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [17]:
playlist.loc[0]['songs']

[373313,
 151080,
 275346,
 696876,
 165237,
 525935,
 457812,
 371709,
 170292,
 438915,
 162569,
 161136,
 310892,
 608704,
 295701,
 127698,
 269427,
 592526,
 110696,
 368214,
 24386,
 104455,
 61612,
 151421,
 416023,
 320447,
 19606]

In [20]:
answers[0]['songs']

[373313,
 151080,
 275346,
 696876,
 165237,
 525935,
 457812,
 371709,
 170292,
 438915,
 162569,
 161136,
 310892,
 608704,
 295701,
 127698,
 269427,
 592526,
 110696,
 368214,
 24386,
 104455,
 61612,
 151421,
 416023,
 320447,
 19606,
 647864,
 583581,
 428196,
 600127,
 239291,
 443007,
 123615,
 542892,
 664555,
 591244,
 399370,
 482876,
 576047,
 430005,
 513889,
 631274,
 426269,
 404828,
 197306,
 622058,
 553610,
 549561,
 24958,
 284179,
 195300,
 77612,
 244396,
 113667,
 286289,
 108084,
 208472,
 428415,
 207806,
 677178,
 523676,
 334119,
 76256,
 432284,
 431773,
 648871,
 576785,
 36908,
 132544,
 397910,
 32278,
 492481,
 48950,
 659616,
 256022,
 593416,
 484821,
 389926,
 125407,
 562491,
 151568,
 3870,
 152819,
 39723,
 632496,
 271601,
 565326,
 67450,
 640721,
 344564,
 320698,
 700842,
 615391,
 592733,
 244596,
 150571,
 663248,
 697793,
 214616]